<a href="https://colab.research.google.com/github/JihenSkima/Computer_Vision_Projects/blob/main/Pencil_sketch_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [17]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import streamlit as st


In [19]:
#Dictionnary to store Pil images
images = {"original":None , "sketch":None}

def open_file():
  filepath = filedialog.askopenfilename() #here the user can choose an image file
  if not filepath: #here if the user didn't choose anything so exit
    return
  img = cv2.imread(filepath) #read the selected image
  display_image(img,original=True)
  sktech_img = convert_to_sketch(img)
  display_image(sketch_img,original=False)





In [5]:
#Conversion to sketch
def convert_to_sketch(img):
  gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#first here we're gonne convert the image to gray scale
  inverted_img = cv2.bitwise_not(gray_img) #to get it negative ( black to white and white to black ( inversé))
  blurred_img = cv2.GaussianBlur(inverted_img,(21,21),sigmaX=0,sigmaY=0)#to smooth it out
  inverted_blur_img = cv2.bitwise_not(blurred_img) #invert to burred image to get back to the contrast
  sketch_img = cv2.divide(gray_img,inverted_blur_img,scale=256.0)#pencil sketch (créer l'effet dessin au crayon)
  return sketch_img

In [20]:
#Displaying the image
def display_image(img,original):
  img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) if original else img
  img_pil = Image.fromarray(img_rgb)
  img_tk = ImageTk.PhotoImage(image=img_pil)

  #Store the PIL image in the dictionary
  if original:
    images["original"] = img_pil
  else:
    images['sketch'] = img_pil

  label = original_image_label if original else sketch_image_label
  label.config(image=img_tk)
  label.image = img_tk


In [21]:
def save_sketch():
  if images['sketch'] is None: #check if there is a sketched image
    messagebox.showerror('Error','No sketch to save ')
    return

  sketch_filepath = filedialog.asksaveasfilename(defaultextension='.png',filetypes=[("PNG files","*.png")])
  if not sketch_filepath:
    return
  #save the PIL image(sketch to the file )
  images['sketch'].save(sketch_filepath,'PNG ')
  messagebox.showinfo("Saved","Sketch saved to {}".format(sketch_filepath))


In [ ]:
app = tk.Tk()
app.title("Pencil sketch converter")

frame = tk.Frame(app)
frame.pack(pady=10,pdax=10)
original_image_label = tk.Label(frame)
original_image_label.grid(row=0,column=0,padx=5,pady=5)
sketch_image_label = tk.Label(frame)
sketch_image_label.grid(row=0,column=1,padx=5,pady=5)

open_button = tk.Frame(btn_frame,text='Open Image',command=open_file)
open_button.grid(row=0,column=0,padx=5)

save_button = tk.Frame(btn_frame,text="Save sketch",command=save_sketch)
save_button.grid(row=0,column=1,padx=5)

app.mainloop()